Outline:

* [1. ERRORS AND EXCEPTIONS](#errors)
* [1.1 Handling Exceptions](#handling)
* [1.2 Defining your own exceptions](#own)
* [1.3 Simple Authorization and Authentification System](#auth)


# 1. ERRORS AND EXCEPTIONS<a class="anchor" id="errors"></a>

## 1.1 Handling Exceptions<a class="anchor" id="handling"></a>

In [2]:
# Python 3 - Object oriented Programming [Page: 105]
def funny_division(anumber):
    try:
        if anumber == 13:
            raise ValueError("13 is an unlucky number")
        return 100 / anumber
    except ZeroDivisionError:
        return "Enter a number other than zero"
    except TypeError:
        return "Enter a numerical value"
    except ValueError:
        print("No, No, not 13!")
        raise
        
for val in (0, "hello", 50.0, 13):
    print("Testing {}:".format(val))
    print(funny_division(val))

Testing 0:
Enter a number other than zero
Testing hello:
Enter a numerical value
Testing 50.0:
2.0
Testing 13:
No, No, not 13!


ValueError: 13 is an unlucky number

---

In [4]:
# Python 3 - Object oriented Programming [Page: 106]
import random
some_exceptions = [ValueError, TypeError, IndexError, None]

try:
    choice = random.choice(some_exceptions)
    print("raising {}".format(choice))
    if choice:
        raise choice("An error")
except ValueError:
    print("Caught a ValueError")
except TypeError:
    print("Caught a TypeError")
except Exception as e:
    print("Caught some other error: %s" % ( e.__class__.__name__))
else:
    print("This code called if there is no exception")
finally:
    print("This cleanup code is always called")

raising <class 'ValueError'>
Caught a ValueError
This cleanup code is always called


## 1.2 Defining your own exceptions<a class="anchor" id="own"></a>

In [5]:
# Python 3 - Object oriented Programming [Page: 110]
class InvalidWithdrawal(Exception):
    def __init__(self, balance, amount):
        super().__init__("account doesn't have ${}".format(amount))
        
        self.amount = amount
        self.balance = balance
        
    def overage(self):
        return self.amount - self.balance

try:
    raise InvalidWithdrawal(25,50)
except InvalidWithdrawal as e:
    print("I'm sorry, but your withdrawal is "
            "more than your balance by "
            "${}".format(e.overage()))

I'm sorry, but your withdrawal is more than your balance by $25


---

In [ ]:
#Incomplete Inventory Example
# Python 3 - Object oriented Programming [Page: 112]
class Inventory:
    def lock(self, item_type):
        ''' Select the type of item that is going to be manipulated. This method will lock
            the item. This prevents selling the same item to two different customers
        '''
        pass
    
    def unlock(self, item_type):
        ''' Release the given type so that other customer can access it.
        '''
        pass
    
    def purchase(self, item_type):
        ''' If the item is not locked, does not exists, out of stock, raise an exception.
            If the item is available, subtract one item and return the number of items left.'''
        pass
    
item_type = 'widget'
inv = Inventory()
inv.lock(item_type)
try:
    num_left = inv.purchase(item_type)
except InvalidItemType:
    print("Sorry, we don't sell {}".format(item_type))
except OutOfStock:
    print("Sorry, that item is out of stock.")
else:
    print("Purchase complete. There are "
            "{} {}s left".format(num_left, item_type))
finally:
    inv.unlock(item_type)
    

## 1.3 Simple Authorization and Authentification System<a class="anchor" id="auth"></a>

In [40]:
# Python 3 - Object oriented Programming [Page: 11]
import random
class Authenticator:
    def __init__(self):
        '''Construct an authenticator to manage users loggin in and out.'''
        self.users = {}
    
    def add_user(self, username, password):
        if username in self.users:
            print("UsernameAlreadyExists[{}]".format(username))
        
        if len(password) < 6:
            print("PasswordTooShort[{}]".format(username))

        self.users[username] = password

In [41]:
auth = Authenticator()
auth.add_user("joe", "joepass")
auth.add_user("mirza", "mirzapass")
auth.add_user("robert", "robertpass")

In [42]:
auth.users

{'joe': 'joepass', 'mirza': 'mirzapass', 'robert': 'robertpass'}

In [43]:
auth.add_user("joe", "joenewpass")

UsernameAlreadyExists[joe]


In [44]:
auth.add_user("mirza", "mir")

UsernameAlreadyExists[mirza]
PasswordTooShort[mirza]


In [1]:
# Python 3 - Object oriented Programming [Page: 11]
import hashlib
class User:
    def __init__(self, username, password):
        '''Create a new User object. The password will be encrypted before storing'''
        self.username = username
        self.password = self._encrypt_pw(password)
        self.is_logged_in = False
        
    def _encrypt_pw(self, password):
        '''Enrypt the password with the username and sha digest.'''
        hash_string = (self.username + password)
        hash_string = hash_string.encode('utf8')
        return hashlib.sha256(hash_string).hexdigest()
    
    def check_password(self, password):
        '''Return True if the password is valid for user. false otherwise.'''
        encrypted = self._encrypt_pw(password)
        return encrypted == self.password


class AuthException(Exception):
    def __init__(self, username, user=None):
        super().__init__(username, user)
        self.username = username
        self.user = user

class UsernameAlreadyExists(AuthException):
    pass

class PasswordTooShort(AuthException):
    pass

class InvalidUsername(AuthException):
    pass

class InvalidPassword(AuthException):
    pass
    
class Authenticator:
    def __init__(self):
        '''Construct an authenticator to manage users loggin in and out.'''
        self.users = {}
    
    def add_user(self, username, password):
        if username in self.users:
            raise UsernameAlreadyExists(username)
        
        if len(password) < 6:
            raise PasswordTooShort(username)

        self.users[username] = User(username, password)
    
    def login(self, username, password):
        try:
            user = self.users[username]
        except KeyError:
            raise InvalidUsername(username)
            
        if not user.check_password(password):
            raise InvalidPassword(username, user)
            
        user.is_logged_in = True
        return True
    
    def is_logged_in(self, username):
        if username in self.users:
            return self.users[username].is_logged_in
        return False

class PermissionError(Exception):
    pass

class Authorizor:
    def __init__(self, authenticator):
        self.authenticator = authenticator
        self.permissions = {}
        
    def add_permission(self, perm_name):
        '''Create a new permission that users can be added to'''
        try:
            perm_set = self.permissions[perm_name]
        except KeyError:
            self.permissions[perm_name] = set()
        else:
            raise PermissionError("Permission Exists")
    
    def permit_user(self, perm_name, username):
        '''Grant the given permission to the user'''
        try:
            perm_set = self.permissions[perm_name]
        except KeyError:
            raise PermissionError("Permission does not exist")
        else:
            if username not in self.authenticator.users:
                raise InvalidUsername(username)
            perm_set.add(username)
            
    def check_permission(self, perm_name, username):
        if not self.authenticator.is_logged_in(username):
            raise NotLoggedInError(username)
        try:
            perm_set = self.permissions[perm_name]
        except KeyError:
            raise PermissionError("Permission does not exist")
        else:
            if username not in perm_set:
                raise NotPermittedError(username)
            else:
                return True

class NotLoggedInError(AuthException):
    pass

class NotPermittedError(AuthException):
    pass

authenticator = Authenticator()
authorizor = Authorizor(authenticator)

In [2]:
authenticator.add_user("joe", "joepassword")

In [3]:
authorizor.add_permission("paint")

In [4]:
authorizor.permissions

{'paint': set()}

In [5]:
authorizor.add_permission("paint")

PermissionError: Permission Exists

In [6]:
authorizor.check_permission("paint", "joe")

NotLoggedInError: ('joe', None)

In [7]:
authenticator.is_logged_in("joe")

False

In [8]:
authenticator.login("joe", "mirzapass")

InvalidPassword: ('joe', <__main__.User object at 0x7f2a21efacf8>)

In [9]:
authenticator.login("joe", "joepassword")

True

In [10]:
authenticator.is_logged_in("joe")

True

In [11]:
authorizor.check_permission("paint", "joe")

NotPermittedError: ('joe', None)

In [12]:
authorizor.permit_user("paint", "joe")

In [13]:
authorizor.check_permission("paint", "joe")

True

In [ ]:
class Editor:
    def __init__(self):
        self.username = None
        self.menu_map = {
            "login": self.login,
            "test": self.test,
            "change": self.change,
            "quit": self.quit}
        
    def login(self):
        logged_in = False
        while not logged_in:
            username = input("username: ")
            password = input("password: ")
            try:
                logged_in = authenticator.login(username, password)
            except InvalidUsername:
                print("Sorry, that username does not exist")
            except InvalidPassword:
                print("Sorry, incorrect password")
            else:
                self.username = username
                
    def is_permitted(self, permission):
        try:
            authorizor.check_permission(permission, self.username)
        except NotLoggedInError as e:
            print("{} is not logged in".format(e.username))
            return False
        except NotPermittedError as e:
            print("{} cannot {}".format(e.username, permission))
            return False
        else:
            return True
        
    def test(self):
        if self.is_permitted("test program"):
            print("Testing program now...")
            
    def change(self):
        if self.is_permitted("change program"):
            print("Changing program now...")
            
    def quit(self):
        raise SystemExit()
        
    def menu(self):
        try:
            answer = ""
            while True:
                print("""
                        Please enter a command:
                        \tlogin\tLogin
                        \ttest\tTest the program
                        \tchange\tChange the program
                        \tquit\tQuit
                        """)
                answer = input("enter a command: ").lower()
                try:
                    func = self.menu_map[answer]
                except KeyError:
                    print("{} is not a valid option".format(answer))
                else:
                    func()
        finally:
            print("Thank you for testing the auth module")

editor = Editor().menu()


                        Please enter a command:
                        	login	Login
                        	test	Test the program
                        	change	Change the program
                        	quit	Quit
                        
